In [1]:
!nvidia-smi

Sun Mar 21 16:55:31 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    22W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# Install all needed packages
!pip install pytorch-lightning
!pip install git+https://github.com/kornia/kornia
!pip uninstall -y tensorflow  # Latest tensorflow breaks logging in pytorch-lightning
!pip install tensorboard

     |████████████████████████████████| 829kB 5.9MB/s 
     |████████████████████████████████| 112kB 39.1MB/s 
     |████████████████████████████████| 829kB 28.5MB/s 
     |████████████████████████████████| 276kB 48.6MB/s 
     |████████████████████████████████| 1.3MB 37.7MB/s 
     |████████████████████████████████| 296kB 48.6MB/s 
     |████████████████████████████████| 143kB 48.2MB/s 
  Created wheel for future: filename=future-0.18.2-cp37-none-any.whl size=491058 sha256=e2250fca6379cd92f9f50504a3a346288400825d5704c1b479dccae4423af068
  Stored in directory: /root/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
  Created wheel for PyYAML: filename=PyYAML-5.3.1-cp37-cp37m-linux_x86_64.whl size=44620 sha256=ed4c97a2d819adc58defe8a5294385b7654af95ce583ed415b6211e55c815404
  Stored in directory: /root/.cache/pip/wheels/a7/c1/ea/cf5bd31012e735dc1dfea3131a2d5eae7978b251083d6247bd
Successfully built future PyYAML
  Found existing installation: future 0.16.0
   

In [3]:
import os
import pathlib

In [4]:
# Put here path to the directory where CUFEDS's input.zip, ref.zip and CUFED5.zip are located. Instruction of how to get all files are in README
gd_data_dirpath = pathlib.Path('drive/MyDrive/skoltech/data') 

input_zippath = gd_data_dirpath / 'input.zip'
ref_zippath = gd_data_dirpath / 'ref.zip'
gt_zippath = gd_data_dirpath / 'CUFED5.zip'

data_dirpath = pathlib.Path('CUFED')
train_data_dirpath = pathlib.Path('CUFED') / 'train'
test_data_dirpath = pathlib.Path('CUFED') / 'test'

In [5]:
# Prepare the data

!mkdir -p CUFED/train
!mkdir -p CUFED/test

!unzip -uq $input_zippath -d $train_data_dirpath
!unzip -uq $ref_zippath -d $train_data_dirpath
!unzip -uq $gt_zippath -d $test_data_dirpath

In [6]:
# Close our repo

!rm -rf train-suite
!git clone https://github.com/theleokul/train-suite.git

Cloning into 'train-suite'...
remote: Enumerating objects: 538, done.
remote: Counting objects: 100% (538/538), done.
remote: Compressing objects: 100% (365/365), done.
remote: Total 538 (delta 331), reused 359 (delta 152), pack-reused 0
Receiving objects: 100% (538/538), 423.78 KiB | 3.07 MiB/s, done.
Resolving deltas: 100% (331/331), done.


In [10]:
# Training baseline (for 2 epochs with reconstruction loss only)
# In order to reproduce different experiments, change the config (all are available in our repo)
!python train-suite/main.py --config train-suite/config/train/exp_4_2/ttsr_warmup_colab.yml -g 0 \
    -L 'logs'  # Path for logs and checkpoints

GPU available: True, used: True
TPU available: None, using: 0 TPU cores

  | Name  | Type               | Params
---------------------------------------------
0 | loss  | ReconstructionLoss | 0     
1 | model | TTSR               | 4.0 M 
---------------------------------------------
3.4 M     Trainable params
555 K     Non-trainable params
4.0 M     Total params
15.846    Total estimated model params size (MB)
Epoch 0:   0% 0/1445 [00:00<?, ?it/s] Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fc84aeff8c0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1324, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1297, in _shutdown_workers
    w.join(timeout=_utils.MP_STATUS_CHECK_INTERVAL)
  File "/usr/lib/python3.7/multiprocessing/process.py", line 140, in join
    res = self._popen.wait(timeout)
  File "/usr/lib/

In [14]:
!python train-suite/main.py --config train-suite/config/train/exp_9/ttsr_colab.yml -g 0 \
    \ # Here you should put path to baseline recieved from the previous cell. It is stored in the logs.
    -b 'drive/MyDrive/skoltech/exp_9/lightning_logs/version_0/checkpoints/epoch=1-a_psnr=25.2712-a_ssim=0.7421.ckpt' \
    -L 'logs'  # Path for logs and checkpoints

Loaded: drive/MyDrive/skoltech/exp_9/lightning_logs/version_0/checkpoints/epoch=1-a_psnr=23.2778-a_ssim=0.6304.ckpt
GPU available: True, used: True
TPU available: None, using: 0 TPU cores

  | Name          | Type          | Params
------------------------------------------------
0 | model         | TTSR          | 7.3 M 
1 | discriminator | Discriminator | 17.8 M
2 | vgg19         | Vgg19         | 12.9 M
------------------------------------------------
24.6 M    Trainable params
13.5 M    Non-trainable params
38.1 M    Total params
152.206   Total estimated model params size (MB)
Epoch 0:  91% 1320/1445 [11:47<01:07,  1.86it/s, loss=-12.2, v_num=1, a_psnr=22.00, a_ssim=0.573]
Validating: 0it [00:00, ?it/s]
Validating:   0% 0/126 [00:00<?, ?it/s]
Epoch 0:  93% 1340/1445 [11:51<00:55,  1.88it/s, loss=-12.2, v_num=1, a_psnr=22.00, a_ssim=0.573]
Epoch 0:  94% 1360/1445 [11:55<00:44,  1.90it/s, loss=-12.2, v_num=1, a_psnr=22.00, a_ssim=0.573]
Epoch 0:  96% 1380/1445 [11:58<00:33,  1.92it/

In [24]:
# Get predictions on validation dataset
!python train-suite/main.py --config train-suite/config/test/exp_9/ttsr_colab.yml -g 0 \
    --modes predict \
    -m 'drive/MyDrive/skoltech/exp_9/lightning_logs/version_1/checkpoints/epoch=49-a_psnr=22.3143-a_ssim=0.5979.ckpt' \
    -O '/path/for/logs/plots/checkpoints'  # Output directory for predictions

Loaded: drive/MyDrive/skoltech/exp_9/lightning_logs/version_1/checkpoints/epoch=49-a_psnr=22.3143-a_ssim=0.5979.ckpt
100% 126/126 [01:04<00:00,  1.97it/s]


In [25]:
# Estimate metrics
!python train-suite/main.py --config train-suite/config/test/exp_9/ttsr_colab.yml -g 0 \
    --modes test \
    -m 'drive/MyDrive/skoltech/exp_9/lightning_logs/version_1/checkpoints/epoch=49-a_psnr=22.3143-a_ssim=0.5979.ckpt'

Loaded: drive/MyDrive/skoltech/exp_9/lightning_logs/version_1/checkpoints/epoch=49-a_psnr=22.3143-a_ssim=0.5979.ckpt
GPU available: True, used: True
TPU available: None, using: 0 TPU cores
Testing: 100% 126/126 [00:29<00:00,  4.34it/s]
--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'a_psnr': 22.3143253326416, 'a_ssim': 0.5979177344216595}
--------------------------------------------------------------------------------
